<center>
<img src="https://www.pragma.com.co/hubfs/pragma_theme2021/images/Header%20Logo.svg" alt="pragma" title="pragma">

# PRUEBA TECNICA

    Presentado por: HUMBERTO FRANCO OSORIO 

    Cargo al que aplica: INGENIERO DE DATOS
</center>

## **PIPELINE**

***Nota**: para una correcta ejecución del notebook ejecute cada bloque en orden*

### Importar librerías necesarias

In [1]:
## LIBRERIAS
import pandas as pd         # Necesaria para lectura de archivo csv
import mysql.connector      # Necesaria para conectar con la base de datos
import datetime             # Necesaria para el manejo del formato de la fecha que se ingresa en la base de datos
from IPython.display import clear_output    # limpiar salida de celdas
import time                                 # utilizada para crear una pausa    

### Funciones para creación registros y consultas en base de datos

In [5]:
# FUNCIONES PARA EL ETL

def dimUserSQL ( data: int, instruction: str, conection: object ) -> tuple:
    if instruction == 'insert':
        query = f"INSERT INTO dim_user (user_id) VALUES ({data})"
        cursor.execute(query)
        conection.commit()
    
    query = f"SELECT * FROM dim_user WHERE user_id = {data};"
    
    cursor.execute(query)
    
    return cursor.fetchone()

def dimTimeSQL ( date: datetime, instruction: str, conection: object ) -> tuple:
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    
    if instruction == 'insert':
        month = date.month
        semester = int(month / 6) + 1
        trimester = int(month / 3) + 1
        year = date.year
        
        query = f"INSERT INTO dim_time (date, year, trimester, semester, month) VALUES ('{date}', {year}, {trimester}, {semester}, {month})"
        cursor.execute(query)
        conection.commit()
    
    
    query = f"SELECT time_key, date FROM dim_time WHERE date = '{date}';"
    cursor.execute(query)
    
    return cursor.fetchone()

def hVentasSQL( user_key: int, time_key:int, price: float) -> None:
    query = f"INSERT INTO h_ventas (user_key, time_key, price) VALUES ({user_key}, {time_key}, {price})"
    cursor.execute(query)
    conection.commit()


def consultKeys() -> list:
    query = "SELECT * FROM dim_user;"
    cursor.execute(query)
    user = cursor.fetchall()
    
    query = "SELECT time_key, date FROM dim_time;"
    cursor.execute(query)
    time = cursor.fetchall()
    
    return user, time
    


### PIPELINE principal

**1.** Configuración conexión base de datos

En los siguientes campos configurar los campos necesarios para conexión a base de datos en MySQL.

* **user:** usario para acceso a MySQL (phpMyAdmin).
* **password:** contraseña de acceso.
* **host:** dirección de conexión a base de datos MySQL (para este caso se accede a la base de datos por medio del localhost al usar XAMPP).
* **dataBaseName:** nombre de la base de datos a la que se conectara.

In [2]:
dbSettings  = {
    'user': 'root',
    "password": '',
    "host": 'localhost',
    "dataBaseName": "ventas_pragma"
}

**2.**  Configuración lectura de archivos

En los siguientes campos se debe proveer la información para la correcta lectura de los archivos CSV teniendo las siguientes consideraciones:
* **quantity:** numero entero que corresponde al numero de archivos que se procesaran en el pipline , para el caso del ejercicio planteado son 5 archivos. (*Nota: debe ser un numero mayos o igual a 1*)
* **folder:** Se configura la dirección de la carpeta donde se encuentran los archivos CSV. (*Nota: Si los archivos están en la misma carpeta que el presente notebook, esta campo se deja en blanco (‘’)*)
* **nameFile:** si se trata de 1 solo archivo se debe colocar el nombre del archivo con extensión, ejemplo: title.csv \
Si se trata de varios archivos, el nombre de los archivos debe ser igual y al final un número que será el identificador del orden de apertura de archivos, ejemplo: title1.csv, title2.csv, title3.csv, etc… \
Y para estos casos se deberá colocar en este campo el nombre del archivo de l siguiente manera *title{ }.csv*.




In [3]:
file = {
    'quantity': 5,
    'folder': 'data',
    'nameFile': '2012-{}.csv'
}

**3.** uso del PIPELINE

In [6]:
# CODIGO DE PIPELINE

try:
    conection = mysql.connector.connect(              # Abrir la conexion con la base de datos
        user=dbSettings.get('user'),
        password=dbSettings.get('password'),
        host=dbSettings.get('host'),
        database=dbSettings.get('dataBaseName')
    )

    contRow = 0
    minValue = None
    maxValue = 0
    addPrice = 0

    if(conection.is_connected()):
        # Crear un cursor para la DB
        cursor = conection.cursor(buffered=True)

        # Se consultan las keys de los registros existentes en la DB
        userRegister, timeRegister = consultKeys()

        # Recorre los archivo csv en la carpeta data
        for i in range(file.get('quantity')):
            # Leer el archivo csv
            df = pd.read_csv(
                f"{ file.get('folder') }/{ file.get('nameFile').format(i+1) }")

            prt = f"""► Archivo Cargado: { file.get('nameFile').format(i+1) }
            Cantidad registros cargados:          {df.shape[0]}         
            Cantidad registros con valores nulos: {df.isnull().any(axis=1).sum()}"""                                                                      # Verificar si hay valores nulos

            df.dropna(inplace=True)

            for index, row in df.iterrows():                                                                # Recorre el dataframe
            
                row.update({                                                                                # Nuevo formato a la fecha
                    'timestamp': datetime.datetime.strptime(row.get('timestamp'), '%m/%d/%Y').strftime('%Y-%m-%d'),         
                })
                                                
                user = list(filter(lambda reg: reg[1] == row.get('user_id'), userRegister))                             # Buscar si registro de usuario ya esta en la base de datos
                timestamp = list(filter(lambda reg: reg[1].strftime('%Y-%m-%d') == row.get('timestamp'), timeRegister))      # Buscar si registro de fecha ya esta en la base de datos
                
                if len(user) == 0:                                                                                 # Si no existe registro de usuario se inserta
                    user = dimUserSQL(row.get('user_id'), 'insert', conection)                                     # Insertar registro de usuario
                    userRegister.append(user)                                                                      # Agregar registro de usuario a la lista de registros existentes
                else:                                                                     
                    user = user[0]
                    
                if len(timestamp) == 0:                                                                                 # Si no existe registro de usuario se inserta
                    timestamp = dimTimeSQL(row.get('timestamp'), 'insert', conection)                                   # Insertar registro de tiempo            
                    timeRegister.append(timestamp)                                                                      # Agregar registro de tiempo a la lista de registros existentes
                else:                                                                     
                    timestamp = timestamp[0]
                
                hVentasSQL(user[0], timestamp[0], row.get('price'))                                                    # Insertar registro de venta
                    
                if( minValue is None ):
                    minValue = row.get('price')
                
                contRow += 1                                                                                      # Contador de registros
                minValue = float(row.get('price')) if float(row.get('price')) < minValue else minValue            # Minimo valor de venta
                maxValue = float(row.get('price')) if float(row.get('price')) > maxValue else maxValue            # Maximo valor de venta
                addPrice += float(row.get('price'))                                                                 # Suma de valores de venta
                
                # time.sleep(0.5)
                clear_output(wait=True)         
                print(f"""{prt}
► Reporte:
    |- Cantidad de registros cargados:          {contRow}
    |- Price:
        |- Minimo: {minValue}
        |- Maximo: {maxValue}
        |- Medio: {addPrice / contRow}""")
                
            print("")
            print("----------------------------------------------------")

except mysql.connector.Error as error:
    # se genera error si no se puede conectar a la DB
    print("Error al conectar a la base de datos {}".format(error))

finally:
    if conection.is_connected():
        # Se cierra la conexion con la DB
        conection.close()
        print("Carga de datos finalizada")

        registerNumbers = contRow
        average = addPrice / contRow if contRow != 0 else 0


► Archivo Cargado: 2012-5.csv
            Cantidad registros cargados:          31         
            Cantidad registros con valores nulos: 0
► Reporte:
    |- Cantidad de registros cargados:          139
    |- Price:
        |- Minimo: 10.0
        |- Maximo: 100.0
        |- Medio: 57.884892086330936

----------------------------------------------------
Carga de datos finalizada


### COMPROBACIÓN DE RESULTADOS

#### **1.** Estadísticas obtenidas en el pipeline

In [7]:
# RESUMEN DATOS COBTENIDOS EN PROCESO DE CARGA DE DATOS ENE BASE DE DATOS
print(f"""
|- Cantidad de registros cargados:          {registerNumbers}
|- Price:
    |- Minimo: {minValue}
    |- Maximo: {maxValue}
    |- Medio: {average}""")



|- Cantidad de registros cargados:          139
|- Price:
    |- Minimo: 10.0
    |- Maximo: 100.0
    |- Medio: 57.884892086330936


#### **2.** Estadisticas al consultar base de datos

In [8]:
# CONSULTA DATOS CARGADOS EN BASE DE DATOS

dataBaseName = "ventas_pragma"

try:
    conection = mysql.connector.connect(              # Abrir la conexion con la base de datos
        user=dbSettings.get('user'),
        password=dbSettings.get('password'),
        host=dbSettings.get('host'),
        database=dbSettings.get('dataBaseName')
    )
    
    if conection.is_connected():                                                                            # Se establece conexion con base de datos se ejecuta el pipeline
        cursor = conection.cursor( buffered= True)
        
        query = "SELECT COUNT(*), AVG(price), MIN(price), MAX(price) FROM h_ventas;"
        cursor.execute(query)
    
        data = cursor.fetchone()
        
        print(f"""
|- Cantidad de registros cargados:          {data[0]}
|- Price:
    |- Minimo: {data[2]}
    |- Maximo: {data[3]}
    |- Medio: { round(data[1],2)}""")
        
    
    
except mysql.connector.Error as error:
    print("Error al conectar a la base de datos {}".format(error))                                          #se genera error si no se puede conectar a la DB
    exit(1)

finally:
    if conection.is_connected():
        conection.close()                                                                                    # Se cierra la conexion con la DB


|- Cantidad de registros cargados:          139
|- Price:
    |- Minimo: 10
    |- Maximo: 100
    |- Medio: 57.88


### **3.** Ejecución de archivo "validation.csv"

**A.** Configuración archivo(s)

In [9]:
file = {
    'quantity': 1,
    'folder': 'data',
    'nameFile': 'validation.csv'
}

**B.** Ejecución PIPELINE

In [10]:
# CODIGO DE PIPELINE

try:
    conection = mysql.connector.connect(              # Abrir la conexion con la base de datos
        user=dbSettings.get('user'),
        password=dbSettings.get('password'),
        host=dbSettings.get('host'),
        database=dbSettings.get('dataBaseName')
    )

    contRow = 0
    minValue = None
    maxValue = 0
    addPrice = 0

    if(conection.is_connected()):
        # Crear un cursor para la DB
        cursor = conection.cursor(buffered=True)

        # Se consultan las keys de los registros existentes en la DB
        userRegister, timeRegister = consultKeys()

        print(file.get('quantity'))
        # Recorre los archivo csv en la carpeta data
        for i in range(file.get('quantity')):
            # Leer el archivo csv
            df = pd.read_csv(
                f"{ file.get('folder') }/{ file.get('nameFile').format(i+1) }")

            prt = f"""► Archivo Cargado: { file.get('nameFile').format(i+1) }
            Cantidad registros cargados:          {df.shape[0]}         
            Cantidad registros con valores nulos: {df.isnull().any(axis=1).sum()}"""                                                                      # Verificar si hay valores nulos

            df.dropna(inplace=True)

            for index, row in df.iterrows():                                                                # Recorre el dataframe
            
                row.update({                                                                                # Nuevo formato a la fecha
                    'timestamp': datetime.datetime.strptime(row.get('timestamp'), '%m/%d/%Y').strftime('%Y-%m-%d'),         
                })
                                                
                user = list(filter(lambda reg: reg[1] == row.get('user_id'), userRegister))                             # Buscar si registro de usuario ya esta en la base de datos
                timestamp = list(filter(lambda reg: reg[1].strftime('%Y-%m-%d') == row.get('timestamp'), timeRegister))      # Buscar si registro de fecha ya esta en la base de datos
                
                if len(user) == 0:                                                                                 # Si no existe registro de usuario se inserta
                    user = dimUserSQL(row.get('user_id'), 'insert', conection)                                     # Insertar registro de usuario
                    userRegister.append(user)                                                                      # Agregar registro de usuario a la lista de registros existentes
                else:                                                                     
                    user = user[0]
                    
                if len(timestamp) == 0:                                                                                 # Si no existe registro de usuario se inserta
                    timestamp = dimTimeSQL(row.get('timestamp'), 'insert', conection)                                   # Insertar registro de tiempo            
                    timeRegister.append(timestamp)                                                                      # Agregar registro de tiempo a la lista de registros existentes
                else:                                                                     
                    timestamp = timestamp[0]
                
                hVentasSQL(user[0], timestamp[0], row.get('price'))                                                    # Insertar registro de venta
                    
                if( minValue is None):
                    minValue = row.get('price')
                
                contRow += 1                                                                                      # Contador de registros
                minValue = float(row.get('price')) if float(row.get('price')) < minValue else minValue            # Minimo valor de venta
                maxValue = float(row.get('price')) if float(row.get('price')) > maxValue else maxValue            # Maximo valor de venta
                addPrice += float(row.get('price'))                                                                 # Suma de valores de venta
                
                time.sleep(0.5)
                clear_output(wait=True)         
                print(f"""{prt}
► Reporte:
    |- Cantidad de registros cargados:          {contRow}
    |- Price:
        |- Minimo: {minValue}
        |- Maximo: {maxValue}
        |- Medio: {addPrice / contRow}""")
                
            print("")
            print("----------------------------------------------------")

except mysql.connector.Error as error:
    # se genera error si no se puede conectar a la DB
    print("Error al conectar a la base de datos {}".format(error))

finally:
    if conection.is_connected():
        # Se cierra la conexion con la DB
        conection.close()
        print("Carga de datos finalizada")

        registerNumbers = contRow
        average = addPrice / contRow if contRow != 0 else 0


► Archivo Cargado: validation.csv
            Cantidad registros cargados:          8         
            Cantidad registros con valores nulos: 0
► Reporte:
    |- Cantidad de registros cargados:          8
    |- Price:
        |- Minimo: 11.0
        |- Maximo: 92.0
        |- Medio: 41.75

----------------------------------------------------
Carga de datos finalizada


**C.** Consulta base de datos

In [11]:
# CONSULTA DATOS CARGADOS EN BASE DE DATOS

dataBaseName = "ventas_pragma"

try:
    conection = mysql.connector.connect(              # Abrir la conexion con la base de datos
        user=dbSettings.get('user'),
        password=dbSettings.get('password'),
        host=dbSettings.get('host'),
        database=dbSettings.get('dataBaseName')
    )
    
    if conection.is_connected():                                                                            # Se establece conexion con base de datos se ejecuta el pipeline
        cursor = conection.cursor( buffered= True)
        
        query = "SELECT COUNT(*), AVG(price), MIN(price), MAX(price) FROM h_ventas;"
        cursor.execute(query)
    
        data = cursor.fetchone()
        
        print(f"""
|- Cantidad de registros cargados:          {data[0]}
|- Price:
    |- Minimo: {data[2]}
    |- Maximo: {data[3]}
    |- Medio: { round(data[1],2)}""")
        
    
    
except mysql.connector.Error as error:
    print("Error al conectar a la base de datos {}".format(error))                                          #se genera error si no se puede conectar a la DB
    exit(1)

finally:
    if conection.is_connected():
        conection.close()                                                                                    # Se cierra la conexion con la DB


|- Cantidad de registros cargados:          147
|- Price:
    |- Minimo: 10
    |- Maximo: 100
    |- Medio: 57.01
